# RareCandy Retrain + Calibration

This notebook fits isotonic and logistic calibrators, evaluates calibration quality, tunes confidence thresholds for edge per deployed dollar, and saves calibrated model artifacts for deployment.

In [ ]:
from __future__ import annotations
from pathlib import Path
import json
from datetime import datetime, timezone
import joblib
import pandas as pd
from analysis.calibration_utils import (
    load_export,
    build_calibration_dataset,
    split_train_test,
    fit_calibrators,
    apply_calibrators,
    calibration_metrics,
    calibration_curve_table,
    per_bin_pnl_table,
    threshold_sweep,
)
BASE_DIR = Path.cwd()
EXPORTS_DIR = BASE_DIR / "exports"
OUT_DIR = BASE_DIR / "analysis" / "artifacts" / "calibration"
MODEL_DIR = OUT_DIR / "models"
OUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
print("exports:", EXPORTS_DIR)
print("out:", OUT_DIR)


In [ ]:
# 1) Load export and build calibration dataset
raw_df = load_export(EXPORTS_DIR, "rarecandy_export")
dataset = build_calibration_dataset(raw_df, horizon=1, trading_cost=0.0006)
train_df, test_df = split_train_test(dataset, train_frac=0.7)

print("rows_total:", len(raw_df))
print("rows_used:", len(dataset.frame))
print("rows_train:", len(train_df), "rows_test:", len(test_df))
display(dataset.frame.head(3))


In [ ]:
# 2) Fit calibrators and score holdout
calibrators = fit_calibrators(train_df)
scored = apply_calibrators(test_df, calibrators)

y_true = scored["label"].to_numpy(dtype=float)
metric_raw = calibration_metrics(y_true, scored["prob_raw"].to_numpy(dtype=float), n_bins=10)
metric_iso = calibration_metrics(y_true, scored["prob_isotonic"].to_numpy(dtype=float), n_bins=10)
metric_log = calibration_metrics(y_true, scored["prob_logistic"].to_numpy(dtype=float), n_bins=10)

metrics_df = pd.DataFrame([
    {"model": "raw", **metric_raw},
    {"model": "isotonic", **metric_iso},
    {"model": "logistic", **metric_log},
]).sort_values(["ece", "brier"])
display(metrics_df)

selected_model = metrics_df.iloc[0]["model"]
selected_col = {"raw": "prob_raw", "isotonic": "prob_isotonic", "logistic": "prob_logistic"}[selected_model]
print("selected_model:", selected_model)


In [ ]:
# 3) Calibration curve + per-bin P&L + threshold tuning
curve_raw = calibration_curve_table(y_true, scored["prob_raw"], n_bins=10)
curve_iso = calibration_curve_table(y_true, scored["prob_isotonic"], n_bins=10)
curve_log = calibration_curve_table(y_true, scored["prob_logistic"], n_bins=10)

bin_pnl = per_bin_pnl_table(scored, selected_col, n_bins=10)
thresholds = threshold_sweep(scored, selected_col, threshold_min=0.50, threshold_max=0.95, threshold_step=0.01, min_signals=20)

display(bin_pnl)
display(thresholds.head(10))

best_threshold = thresholds.iloc[0].to_dict() if not thresholds.empty else None
print("best_threshold:", best_threshold)


In [ ]:
# 4) Save calibrated artifacts
joblib.dump(calibrators["isotonic"], MODEL_DIR / "isotonic_calibrator.joblib")
joblib.dump(calibrators["logistic"], MODEL_DIR / "logistic_calibrator.joblib")
(MODEL_DIR / "selected_model.txt").write_text(selected_model + "\n", encoding="utf-8")

curve_raw.to_csv(OUT_DIR / "calibration_curve_raw.csv", index=False)
curve_iso.to_csv(OUT_DIR / "calibration_curve_isotonic.csv", index=False)
curve_log.to_csv(OUT_DIR / "calibration_curve_logistic.csv", index=False)
bin_pnl.to_csv(OUT_DIR / "per_bin_pnl.csv", index=False)
thresholds.to_csv(OUT_DIR / "threshold_sweep.csv", index=False)

report = {
    "generated_at_utc": datetime.now(timezone.utc).isoformat(),
    "selected_model": selected_model,
    "selected_model_metrics": metrics_df.iloc[0].to_dict(),
    "best_threshold": best_threshold,
    "rows_test": int(len(scored)),
}
(OUT_DIR / "calibration_report_from_notebook.json").write_text(json.dumps(report, indent=2), encoding="utf-8")

print("Saved models and calibration artifacts under:", OUT_DIR)


In [ ]:
# 5) Optional: run standalone report + alerts scripts from notebook
# !python3 analysis/calibration_report.py
# !python3 analysis/check_calibration_alerts.py
